নিম্নলিখিত নোটবুকগুলি চালানোর জন্য, যদি আপনি এখনও না করে থাকেন, তাহলে আপনাকে .env ফাইলে `OPENAI_API_KEY` হিসেবে openai কী সেট করতে হবে।


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

পরবর্তী, আমরা এমবেডিং ইনডেক্সটি একটি প্যান্ডাস ডেটাফ্রেমে লোড করব। এমবেডিং ইনডেক্সটি একটি JSON ফাইলে সংরক্ষিত আছে যার নাম `embedding_index_3m.json`। এমবেডিং ইনডেক্সে ২০২৩ সালের অক্টোবরের শেষ পর্যন্ত প্রতিটি ইউটিউব ট্রান্সক্রিপ্টের এমবেডিংস রয়েছে।


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

পরবর্তী ধাপে, আমরা `get_videos` নামে একটি ফাংশন তৈরি করব যা কুয়েরির জন্য এমবেডিং ইনডেক্স অনুসন্ধান করবে। ফাংশনটি কুয়েরির সাথে সবচেয়ে মিল থাকা শীর্ষ ৫টি ভিডিও ফেরত দেবে। ফাংশনটি নিম্নরূপ কাজ করে:

1. প্রথমে, এমবেডিং ইনডেক্সের একটি কপি তৈরি করা হয়।
2. এরপর, OpenAI এমবেডিং API ব্যবহার করে কুয়েরির এমবেডিং গণনা করা হয়।
3. তারপর এমবেডিং ইনডেক্সে `similarity` নামে একটি নতুন কলাম তৈরি করা হয়। `similarity` কলামে কুয়েরি এমবেডিং এবং প্রতিটি ভিডিও সেগমেন্টের এমবেডিংয়ের মধ্যে কোসাইন সাদৃশ্য থাকে।
4. এরপর, `similarity` কলাম দ্বারা এমবেডিং ইনডেক্স ফিল্টার করা হয়। এমবেডিং ইনডেক্স শুধুমাত্র সেই ভিডিওগুলো অন্তর্ভুক্ত করতে ফিল্টার করা হয় যাদের কোসাইন সাদৃশ্য ০.৭৫ বা তার বেশি।
5. অবশেষে, `similarity` কলাম দ্বারা এমবেডিং ইনডেক্স সাজানো হয় এবং শীর্ষ ৫টি ভিডিও ফেরত দেওয়া হয়।


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

এই ফাংশনটি খুবই সহজ, এটি কেবল সার্চ কুয়েরির ফলাফলগুলি প্রিন্ট করে।


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. প্রথমে, এমবেডিং ইনডেক্স একটি প্যান্ডাস ডেটাফ্রেমে লোড করা হয়।
2. পরবর্তীতে, ব্যবহারকারীকে একটি প্রশ্ন প্রবেশ করার জন্য অনুরোধ করা হয়।
3. তারপর `get_videos` ফাংশনটি কল করা হয় প্রশ্নের জন্য এমবেডিং ইনডেক্স অনুসন্ধান করতে।
4. অবশেষে, `display_results` ফাংশনটি কল করা হয় ফলাফলগুলি ব্যবহারকারীর কাছে প্রদর্শন করতে।
5. এরপর ব্যবহারকারীকে আবার একটি প্রশ্ন প্রবেশ করার জন্য অনুরোধ করা হয়। এই প্রক্রিয়া চলতে থাকে যতক্ষণ না ব্যবহারকারী `exit` প্রবেশ করে।

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.bn.png)

আপনাকে একটি প্রশ্ন প্রবেশ করার জন্য অনুরোধ করা হবে। একটি প্রশ্ন লিখুন এবং এন্টার চাপুন। অ্যাপ্লিকেশনটি প্রশ্নের সাথে সম্পর্কিত ভিডিওগুলির একটি তালিকা প্রদান করবে। অ্যাপ্লিকেশনটি সেই ভিডিওর লিঙ্কও প্রদান করবে যেখানে প্রশ্নের উত্তর অবস্থিত।

এখানে কিছু প্রশ্ন দেওয়া হলো যা আপনি চেষ্টা করতে পারেন:

- Azure Machine Learning কী?
- কনভলিউশনাল নিউরাল নেটওয়ার্কগুলি কীভাবে কাজ করে?
- নিউরাল নেটওয়ার্ক কী?
- আমি কি Azure Machine Learning এর সাথে Jupyter Notebooks ব্যবহার করতে পারি?
- ONNX কী?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ সেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনূদিত হয়েছে। আমরা যথাসাধ্য সঠিকতার চেষ্টা করি, তবে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল নথিটি তার নিজস্ব ভাষায়ই কর্তৃত্বপূর্ণ উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য পেশাদার মানব অনুবাদ গ্রহণ করার পরামর্শ দেওয়া হয়। এই অনুবাদের ব্যবহারে সৃষ্ট কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যার জন্য আমরা দায়ী নই।
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
